In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams
from tqdm import tqdm
from collections import Counter, defaultdict
import re
import random
from copy import copy

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.


In [2]:
tokenizer = RegexpTokenizer(r'\w+')
abc = 'абвгдежзийклмнопрстуфхцчшщъыьэюя '

def tokenize(text, tokenizer=tokenizer):
    text = ''.join([c for c in text if c.lower() in abc])
    return ' '.join(tokenizer.tokenize(text.lower()))


def get_freqs(text, min_freq=0, n_gram=1):
    freqs = dict()
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        if value/len(text) > min_freq:
            freqs[key] = value/len(text)
    return freqs


def generate_mapping(freqs):
    original = list(freqs.keys())
    replacements = np.random.choice(original, replace=False, size=len(freqs))
    mapping = dict()
    for original_char, replacement_char in zip(original, replacements):
        mapping[original_char] = replacement_char
    return mapping


def apply_mapping(text, mapping):
    return ''.join([mapping.get(c, 'ь') for c in text])

# Здесь мы находим ближайший по частотности символ.
# Пробовал использовать просто ранги, но качество хуже.
def get_reverse_mapping(corpus_freqs, text_freqs):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    for text_char, text_freq in text_freqs_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, corpus_freq in corpus_freqs_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_mapping[text_char] = best_char
        corpus_freqs_sorted = [(char, freq) for char, freq in corpus_freqs_sorted if char !=best_char]
        
    return reverse_mapping


def char_accuracy(text1, text2):
    assert len(text1) == len(text2)
    right_chars = 0
    for c1, c2 in zip(text1, text2):
        right_chars += int(c1 == c2)
    return right_chars / len(text1)

In [3]:
np.random.seed(42)

with open('AnnaKarenina.txt' ,'r') as file:
    corpus1 = file.readlines()
    
with open('WarAndPeace.txt' ,'r') as file:
    corpus2 = file.readlines()
    
corpus = corpus1 + corpus2

tokenized_corpus = tokenize(' '.join(corpus))
corpus_freqs = get_freqs(tokenized_corpus, n_gram=1)
mapping = generate_mapping(corpus_freqs)

In [4]:
with open('text.txt' ,'r') as file:
    text = file.readlines()

tokenized_text = tokenize(' '.join(text))
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_freqs(encoded_text)
tokenized_text

'в течение многих часов шерлок холмс сидел согнувшись над стеклянной пробиркой в которой варилось чтото на редкость вонючее голова его была опущена на грудь и он казался мне похожим на странную товцую птицу с тусклыми серыми перьями и черным хохолком итак уотсон сказал он внезапно вы не собираетесь вкладывать свои сбережения в южноафриканские ценные бумаги я вздрогнул от удивления как ни привык я к необычайным способностям холмса это внезапное вторжение в самые тайные мои мысли было совершенно необъяснимым как черт возьми вы об этом узнали спросил я он повернулся на стуле держа в руке дымящуюся пробирку и его глубоко сидящие глаза радостно заблистали признайтесь уотсон что вы совершенно сбиты с толку сказал он признаюсь мне следовало бы заставить вас написать об этом на листочке бумаги и подписаться почему потому что через пять минут вы скажете что все это необычайно просто'

In [5]:
encoded_text

'сьуоромзоьямахзчьрщеасьжовдаыьчадяеьезподьеахмнсжзеиьмщпьеуоыдэммайьгватзвыайьсьыауавайьсщвздаеиьруауаьмщьвопыаеуиьсамлрооьхадасщьохаьт дщьагнбомщьмщьхвнпиьзьамьыщкщдеэьямоьгачашзяьмщьеувщммнльуасънльгузъньеьунеыд язьеов язьговиэязьзьровм яьчачадыаяьзущыьнауеамьеыщкщдьамьсмокщгмаьс ьмоьеатзвщоуоеиьсыдщп сщуиьесазьетовошомзэьсьлшмащфвзыщмеызоьъомм оьтнящхзьэьскпвахмндьауьнпзсдомзэьыщыьмзьгвзс ыьэьыьмоат рщйм яьегаеатмаеуэяьчадяещьюуаьсмокщгмаоьсуавшомзоьсьещя оьущйм оьяазья едзьт даьеасовжоммаьмоатцэемзя яьыщыьровуьсакиязьс ьатьюуаяьнкмщдзьегваездьэьамьгасовмндеэьмщьеундоьповшщьсьвныоьп яэбнлеэьгватзвыньзьохаьхднтаыаьезпэбзоьхдщкщьвщпаеумаькщтдзеущдзьгвзкмщйуоеиьнауеамьруаьс ьеасовжоммаьетзу ьеьуадыньеыщкщдьамьгвзкмщлеиьямоьедопасщдаьт ькщеущсзуиьсщеьмщгзещуиьатьюуаяьмщьдзеуарыоьтнящхзьзьгапгзещуиеэьгарояньгауаяньруаьровокьгэуиьязмнуьс ьеыщшоуоьруаьсеоьюуаьмоат рщймаьгваеуа'

In [6]:
reverse_mapping = get_reverse_mapping(corpus_freqs, text_freqs)
decoded_text = apply_mapping(encoded_text, reverse_mapping)
decoded_text

'р лиыиеси меойсх ытнор цивкод хокмн нсчик нойеурцснб етч нлидкьееош явогсвдош р доловош ртвсконб ылоло ет вичдонлб роеюыии йокорт ийо гпкт ояущиет ет йвучб с ое дтзткнь меи яохожсм ет нлвтееую лорфую ялсфу н лундкпмс нивпмс яивбьмс с ыивепм хохокдом слтд уолное ндтзтк ое реизтяео рп еи ногсвтилинб рдктчпртлб нрос нгивижиесь р южеотъвсдтендси фиеепи гумтйс ь рзчвойеук ол учсркиесь дтд ес явсрпд ь д еиогпытшепм няоногеонльм хокмнт эло реизтяеои рловжиеси р нтмпи лтшепи мос мпнкс гпко норивциеео еиогаьнесмпм дтд ыивл розбмс рп ог элом узеткс нявонск ь ое яоривеукнь ет нлуки чивжт р вуди чпмьщуюнь явогсвду с ийо йкугодо нсчьщси йктзт втчонлео зтгкснлткс явсзетшлинб уолное ыло рп норивциеео нгслп н локду ндтзтк ое явсзетюнб меи нкичортко гп зтнлтрслб ртн етяснтлб ог элом ет кснлоыди гумтйс с яочяснтлбнь яоыиму яолому ыло ыивиз яьлб мсеул рп ндтжили ыло рни эло еиогпытшео явонло'

In [7]:
char_accuracy(tokenized_text, decoded_text)

0.3536723163841808

В общем, так себе разультат :)

## 2. Базовый частотный метод с биграммами


Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [8]:
corpus_freqs_bigram = get_freqs(tokenized_corpus, n_gram=2)
text_freqs_bigram = get_freqs(encoded_text, n_gram=2)

corpus_freqs_sorted = sorted(corpus_freqs_bigram.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs_bigram.items(), key=lambda x: x[1], reverse=True)

In [9]:
def get_reverse_mapping_bigram(corpus_freqs, text_freqs, init_mapping=None):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    if init_mapping is None:
        init_mapping = []
    reverse_mapping = {k: v for k, v in init_mapping}
    
    for i, (text_bigram, text_freq) in enumerate(text_freqs_sorted):

        filtred_freqs = copy(corpus_freqs_sorted)
        
        if text_bigram[0] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[0] == reverse_mapping[text_bigram[0]]]
            
        if text_bigram[1] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[1] == reverse_mapping[text_bigram[1]]]
              
        min_diff = 1
        best_bigram = None
        for bigram, freq in filtred_freqs:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_bigram = bigram
                min_diff = diff
                
        if text_bigram[0] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[0]] = best_bigram[0]
            
        if text_bigram[1] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[1]] = best_bigram[1]

        
    return reverse_mapping

In [10]:
print(tokenized_text)
print('----------------')
print(encoded_text)

в течение многих часов шерлок холмс сидел согнувшись над стеклянной пробиркой в которой варилось чтото на редкость вонючее голова его была опущена на грудь и он казался мне похожим на странную товцую птицу с тусклыми серыми перьями и черным хохолком итак уотсон сказал он внезапно вы не собираетесь вкладывать свои сбережения в южноафриканские ценные бумаги я вздрогнул от удивления как ни привык я к необычайным способностям холмса это внезапное вторжение в самые тайные мои мысли было совершенно необъяснимым как черт возьми вы об этом узнали спросил я он повернулся на стуле держа в руке дымящуюся пробирку и его глубоко сидящие глаза радостно заблистали признайтесь уотсон что вы совершенно сбиты с толку сказал он признаюсь мне следовало бы заставить вас написать об этом на листочке бумаги и подписаться почему потому что через пять минут вы скажете что все это необычайно просто
----------------
сьуоромзоьямахзчьрщеасьжовдаыьчадяеьезподьеахмнсжзеиьмщпьеуоыдэммайьгватзвыайьсьыауавайьсщвздаеиь

In [11]:
reverse_mapping_bigram = get_reverse_mapping_bigram(corpus_freqs_bigram, text_freqs_bigram)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
print(decoded_text)

 оатвтвттонв гтвово   оитон ков нн о тбтно  гв  ит  овобо аткннвв го о кток го ок а о го оотн   ова а овооотбк  а о  влвттог н  оотг океноо   ятвоовоого б ото воконон нонвто  в итновоо аоовв лоа  х ло атх о оа  кненто тоенто то ннтотовтовенов в нк нотаоко  а  во кононо во втно в о еовто  ктоотат  о кнобе оа о   то ктотитвтно олив опотков кттохтвветок ногтоно нбо гв но ао бт нтвтнококовто от еконоковт кевогвено     кв  аннов нн оога о втно в то а оитвтто о онетоаогветон тоне нтокен о   тоитвв овт кнн втненококовтоао  н нто ео кога но нвонто  о  тноно во   тов н новоо а нтобтоиоо оо ктобення л но о кток ототг огн к к о тбняттогнонооооб  ав онокнт аонто отнвогат  о  а  вова о ео   тоитвв о ктаео оа нк о кононо во отнвол  онвто нтб  он океоно ао та о о ово т оа о кога новоонт а вкток ногтото  б т оа  но  втн о  а н ова овтотно на онтв ао ео коитатова о  тога овт кевогв о о  а 


In [12]:
char_accuracy(tokenized_text, decoded_text)

0.04180790960451977

Стало хуже, что неудивительно. Биграмм больше, попасть по частоте в правильные - сложнее. Нужен текст сильно больше.

Попробуем добавить 1-2 раскодированных с помощью посимвольного метода символа как инициализацию.

In [13]:
init_mapping = list(reverse_mapping.items())[:2]
init_mapping

[('ь', ' '), ('а', 'о')]

In [14]:
reverse_mapping_bigram = get_reverse_mapping_bigram(corpus_freqs_bigram, text_freqs_bigram, init_mapping=init_mapping)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
decoded_text

'с такапаа ьпоеал косос  алнок лоньс сабан соепрс ася поб стакньппог словалког с котолог соланося ктото по лабкостя сопукаа еоносо аео втно осрнапо по елрбя а оп кононсь ьпа соло аь по стлоппру тосару стаар с трскнтьа салтьа саляььа а калпть лолонкоь аток ротсоп сконон оп спаноспо ст па совалоатася скнобтсотя ссоа свала апаь с у пооплакопскаа ааппта врьоеа ь снблоепрн от рбаснапаь кок па сластк ь к паовткогпть ссосовпостьь лоньсо ето спаноспоа стол апаа с соьта тогпта ьоа ьтсна втно сосал аппо паовшьспаьть кок калт соняьа ст ов етоь рнпона сслосан ь оп сосалпрнсь по стрна бал о с лрка бтььнрусь словалкр а аео енрвоко сабьнаа еноно лобостпо новнастона сланпогтася ротсоп кто ст сосал аппо сватт с тонкр сконон оп сланпоуся ьпа снабосоно вт ностосатя сос посасотя ов етоь по настокка врьоеа а собсасотясь сокаьр сотоьр кто калан сьтя ьапрт ст ско ата кто сса ето паовткогпо слосто'

In [15]:
char_accuracy(tokenized_text, decoded_text)

0.3898305084745763

Метрики выросли, но прочесть это все так же невозможно :)

## 3. Метод на основе MCMC-семплирования

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


**Метод**

Текст, разбитый на биграммы - это, по сути, марковская цепь. Частота биграмм - вероятность перехода по цепи.

Для обучения перестановки будем считать вероятность порождения именно такого текста как произведение вероятностей всех биграмм, в него входящих. 

Всего перестановок очень много, поэтому сделаем жадный алгоритм, который использует идею MCMC-семплирования.

Алгоритм:

0) Инициализируем перестановки, восстанавливаем текст и считаем на нем $p_{current}$ 

1) Меняем местами пару букв для перестановки

2) Восстанавливаем текст с новой перестановкой и считаем на нем $p_{proposal}$  

3) Принимаем новую перестановку с "вероятностью" $p_{accept} = \frac{p_{proposal}}{p_{current}}$ 

4) Возвращаемся к пункту 1

Мы можем уйти немного не туда и застрять не в том максимуме, поэтому будем делать много попыток и брать лучшую.

In [16]:
def get_freqs_smooth(text, min_freq=0, n_gram=2):
    freqs = dict()
    vocab_len = len(set(text))**n_gram
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        freqs[key] = (value + 1) / (len(text) + vocab_len) # Сглаживаем, чтобы не было нулей
    return freqs

def get_text_proba(text, mapping, freqs, n_gram=2):
    decoded_text = apply_mapping(text, mapping)
    log_proba = 0
    for i in range(len(decoded_text) - n_gram):
        bigram = decoded_text[i: i + n_gram]
        bigram_proba = freqs.get(bigram)
        if bigram_proba is None:
            bigram_proba = 1 / (len(text) + len(abc)**n_gram) # Сглаживаем, чтобы не было нулей
            
        log_proba += np.log(bigram_proba)
    return log_proba
        
def get_reverse_mapping_mcmc(encoded_text, abc_encoded, abc_corpus, freqs_corpus, n_iters=10000, n_trials=10, n_gram=2):

    accept_count = 0
    best_mapping = None
    all_mappings = []
    best_log_likekihood = -np.inf

    for trial in tqdm(range(n_trials), leave=False, position=0, total=n_trials):

        abc_encoded = list(abc_encoded)
        abc_iter = list(abc_corpus)
        reverse_mapping = {k: v for k, v in zip(abc_encoded, abc_iter[:len(abc_encoded)])}
        log_proba_current = get_text_proba(encoded_text, reverse_mapping, freqs_corpus, n_gram=n_gram)

        for i in range(n_iters):
            abc_proposal = abc_iter[:]
            idx1, idx2 = np.random.choice(len(abc_proposal), replace=False, size=2)
            abc_proposal[idx1], abc_proposal[idx2] = abc_proposal[idx2], abc_proposal[idx1]
            reverse_mapping_proposal = {k: v for k, v in zip(abc_encoded, abc_proposal[:len(abc_encoded)])}
            log_proba_proposal = get_text_proba(encoded_text, reverse_mapping_proposal, freqs_corpus, n_gram=n_gram)

            p_accept = np.exp(log_proba_proposal - log_proba_current)

            if p_accept > np.random.rand():
                accept_count += 1
                abc_iter = abc_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likekihood:
            best_log_likekihood = log_proba_current
            best_mapping = reverse_mapping
            
        all_mappings.append(reverse_mapping)


    print(f'Best likelihood: {best_log_likekihood}')        
    print(f'Accept raito: {accept_count / (n_iters * n_trials)}')
    return best_mapping, all_mappings

In [17]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [18]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc, 
    abc_corpus=abc,
    freqs_corpus=freqs_corpus
)

Best likelihood: -4985.946706030881
Accept raito: 0.01208


In [19]:
apply_mapping(encoded_text, best_reverse_mapping)

'в течение многих часов шерлок холмс сидел согнувшись над стеклянной пробиркой в которой варилось чтото на редкость вонючее голова его была опуъена на грудь и он казался мне похожим на странную товцую птицу с тусклыми серыми перьями и черным хохолком итак уотсон сказал он внезапно вы не собираетесь вкладывать свои сбережения в южноафриканские ценные бумаги я вздрогнул от удивления как ни привык я к необычайным способностям холмса это внезапное вторжение в самые тайные мои мысли было совершенно необщяснимым как черт возьми вы об этом узнали спросил я он повернулся на стуле держа в руке дымяъуюся пробирку и его глубоко сидяъие глаза радостно заблистали признайтесь уотсон что вы совершенно сбиты с толку сказал он признаюсь мне следовало бы заставить вас написать об этом на листочке бумаги и подписаться почему потому что через пять минут вы скажете что все это необычайно просто'

Получилось :)

## 4. Расшифруйте сообщение

In [20]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [21]:
message_freqs = get_freqs(message)


In [22]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(message_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_message = ''.join([c for c, _ in message_freqs_sorted])
abc_corpus, abc_message

(' оаеинтслвркмдупягьыбзчйжшхюцэщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [23]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [24]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    message, 
    abc_encoded=abc_message, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus,
    n_iters=10000,
    n_trials=50,
)

Best likelihood: -1236.0627385039725
Accept raito: 0.04439


In [25]:
apply_mapping(message, best_reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешац'

In [26]:
apply_mapping(message, best_reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешац'

Успех :)

In [27]:
original_message = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"
encoded_message = apply_mapping(message, best_reverse_mapping)
char_accuracy(original_message, encoded_message)

0.9347826086956522

## 5. A что если от биграмм перейти к триграммам

Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

**1. Попробуем раскодировать сообщение из пункта 4 триграммами**

In [28]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(message_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_message = ''.join([c for c, _ in message_freqs_sorted])
abc_corpus, abc_message

(' оаеинтслвркмдупягьыбзчйжшхюцэщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [29]:
freqs_corpus_trigramm = get_freqs_smooth(tokenized_corpus, n_gram=3)

In [30]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    message, 
    abc_encoded=abc_message, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=20,
)

Best likelihood: -1726.0526509971126
Accept raito: 0.04171


In [31]:
apply_mapping(message, best_reverse_mapping)

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаф'

In [32]:
original_message = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"
encoded_message = apply_mapping(message, best_reverse_mapping)
char_accuracy(original_message, encoded_message)

0.9956521739130435

Получилось почти идеально, качество выросло. 

**2. Попробуем тексты подлиннее, может что-то пойдет не так**

In [33]:
mapping = generate_mapping(corpus_freqs)
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_freqs(encoded_text)

corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_text = ''.join([c for c, _ in text_freqs_sorted])
abc_corpus, abc_text

(' оаеинтслвркмдупягьыбзчйжшхюцэщфъ', 'бйзу ъдпгшмеотаысвкьяэлчфжринщюхц')

In [34]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus,
    n_gram=2,
    n_iters=10000,
    n_trials=5,
)
print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(tokenized_text, decoded_text))

Best likelihood: -4985.946706030881
Accept raito: 0.00742
-----
гбпукуздубезйлдфбкъ йгбиушмйобфйме б дэумб йлзтгид ьбзъэб пуомвззйчбышйсдшойчбгбойпйшйчбгъшдмй ьбкпйпйбзъбшуэой пьбгйзжкуублймйгъбулйбсамъбйытщузъбзъблштэьбдбйзбоъяъм вбезубыйфйрдебзъб пшъззтжбпйгютжбыпдютб бпт омаедб ушаедбыушьведбдбкушзаебфйфймойебдпъобтйп йзб оъяъмбйзбгзуяъызйбгабзуб йсдшъупу ьбгомъэагъпьб гйдб сушуруздвбгбжрзйъхшдоъз одубюуззаубстеълдбвбгяэшйлзтмбйпбтэдгмуздвбоъобздбышдгаобвбобзуйсакъчзаеб ый йсзй пвебфйме ъбнпйбгзуяъызйубгпйшруздубгб ъеаубпъчзаубейдбеа мдбсамйб йгушиуззйбзуйсцв здеаебоъобкушпбгйяьедбгабйсбнпйебтязъмдб ышй дмбвбйзбыйгушзтм вбзъб птмубэушръбгбштоубэаевщтж вбышйсдшотбдбулйблмтсйойб дэвщдублмъяъбшъэй пзйбяъсмд пъмдбышдязъчпу ьбтйп йзбкпйбгаб йгушиуззйб сдпаб бпймотб оъяъмбйзбышдязъж ьбезуб муэйгъмйбсабяъ пъгдпьбгъ бзъыд ъпьбйсбнпйебзъбмд пйкоубстеълдбдбыйэыд ъпь вбыйкуетбыйпйетбкпйбкушуябывпьбедзтпбгаб оърупубкпйбг убнпйбзуйсакъчзйбышй пй
-----
в течение многих часов шерлок холмс сидел со

In [35]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=5,
)

print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(tokenized_text, decoded_text))

Best likelihood: -6857.856111757894
Accept raito: 0.00908
-----
гбпукуздубезйлдфбкъ йгбиушмйобфйме б дэумб йлзтгид ьбзъэб пуомвззйчбышйсдшойчбгбойпйшйчбгъшдмй ьбкпйпйбзъбшуэой пьбгйзжкуублймйгъбулйбсамъбйытщузъбзъблштэьбдбйзбоъяъм вбезубыйфйрдебзъб пшъззтжбпйгютжбыпдютб бпт омаедб ушаедбыушьведбдбкушзаебфйфймойебдпъобтйп йзб оъяъмбйзбгзуяъызйбгабзуб йсдшъупу ьбгомъэагъпьб гйдб сушуруздвбгбжрзйъхшдоъз одубюуззаубстеълдбвбгяэшйлзтмбйпбтэдгмуздвбоъобздбышдгаобвбобзуйсакъчзаеб ый йсзй пвебфйме ъбнпйбгзуяъызйубгпйшруздубгб ъеаубпъчзаубейдбеа мдбсамйб йгушиуззйбзуйсцв здеаебоъобкушпбгйяьедбгабйсбнпйебтязъмдб ышй дмбвбйзбыйгушзтм вбзъб птмубэушръбгбштоубэаевщтж вбышйсдшотбдбулйблмтсйойб дэвщдублмъяъбшъэй пзйбяъсмд пъмдбышдязъчпу ьбтйп йзбкпйбгаб йгушиуззйб сдпаб бпймотб оъяъмбйзбышдязъж ьбезуб муэйгъмйбсабяъ пъгдпьбгъ бзъыд ъпьбйсбнпйебзъбмд пйкоубстеълдбдбыйэыд ъпь вбыйкуетбыйпйетбкпйбкушуябывпьбедзтпбгаб оърупубкпйбг убнпйбзуйсакъчзйбышй пй
-----
в течение многих часов шерлок холмс сидел со

Качество практически не изменилось.

**3. Попробуем еще одну перестановку, но уже с очень коротким текстом**

In [36]:
short_text = 'это очень короткий текст чтобы проверить гипотезу что совсем маленькие тексты раскодируются последовательностями из двух букв хуже чем из трех'

mapping = generate_mapping(corpus_freqs)
encoded_text = apply_mapping(short_text, mapping)
text_freqs = get_freqs(encoded_text)

corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_text = ''.join([c for c, _ in text_freqs_sorted])
abc_corpus, abc_text

(' оаеинтслвркмдупягьыбзчйжшхюцэщфъ', ' ктбъаьсчхюшрифвмзнеяэлдйожг')

**Биграммы**

In [37]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus,
    n_gram=2,
    n_iters=10000,
    n_trials=5,
)
print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(short_text, decoded_text))

Best likelihood: -786.2069554555712
Accept raito: 0.05292
-----
дкт тюбиш ьтсткьъй кбьак юктяэ фстчбсъкш оъфткбвх юкт атчабр рмзбишьъб кбьакэ смаьтнъсхжкал фтазбнтчмкбзшитаклръ ъв нчхе яхьч ехгб юбр ъв ксбе
-----
жта акори ванатвею товст ктабы зналонети чезатойя кта салсом мупоривео товсты нусваденяътсь засподалутопирастьме ей длях бявл хяго ком ей тнох
-----
0.3873239436619718


**Триграммы**

In [38]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=5,
)

print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(short_text, decoded_text))

Best likelihood: -1094.0068699740987
Accept raito: 0.0475
-----
дкт тюбиш ьтсткьъй кбьак юктяэ фстчбсъкш оъфткбвх юкт атчабр рмзбишьъб кбьакэ смаьтнъсхжкал фтазбнтчмкбзшитаклръ ъв нчхе яхьч ехгб юбр ъв ксбе
-----
это очень короткий текст чтобы проверить хипотезу что совсем маленькие тексты раскогирудтся послеговательностями из гвую букв юуже чем из трею
-----
0.9436619718309859


**4-граммы**

In [39]:
freqs_corpus_fourgramm = get_freqs_smooth(tokenized_corpus, n_gram=4)

In [40]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_fourgramm,
    n_gram=4,
    n_iters=10000,
    n_trials=5,
)

print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(short_text, decoded_text))

Best likelihood: -1448.716050748896
Accept raito: 0.04956
-----
дкт тюбиш ьтсткьъй кбьак юктяэ фстчбсъкш оъфткбвх юкт атчабр рмзбишьъб кбьакэ смаьтнъсхжкал фтазбнтчмкбзшитаклръ ъв нчхе яхьч ехгб юбр ъв ксбе
-----
это очень короткий текст чтобы проверить гипотезу что совсем маленькие тексты раскодируфтся последовательностями из двух букв хуже чем из трех
-----
0.9929577464788732


Получилось просто идеально.

**Наблюдение по n-граммам**

Похоже на то, что триграммы дают хороший прирост качества для декодирования коротких текстов. Для длинных текстов в принципе и так все хорошо.